### libs

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, ttest_rel, ttest_ind_from_stats

### data

In [16]:
import seaborn as sns

df = sns.load_dataset("titanic")
print(f"{df.shape = }")
print(df.head().to_string())

df.shape = (891, 15)
   survived  pclass     sex   age  sibsp  parch     fare embarked  class    who  adult_male deck  embark_town alive  alone
0         0       3    male  22.0      1      0   7.2500        S  Third    man        True  NaN  Southampton    no  False
1         1       1  female  38.0      1      0  71.2833        C  First  woman       False    C    Cherbourg   yes  False
2         1       3  female  26.0      0      0   7.9250        S  Third  woman       False  NaN  Southampton   yes   True
3         1       1  female  35.0      1      0  53.1000        S  First  woman       False    C  Southampton   yes  False
4         0       3    male  35.0      0      0   8.0500        S  Third    man        True  NaN  Southampton    no   True


In [5]:
df.groupby(["who"], dropna=False, observed=False).agg(
    **{
        "n": ("who", "size"),
        "survived_rate": ("survived", "mean"),
    }
)

,n,survived_rate
who,,
child,83,0.590361
man,537,0.163873
woman,271,0.756458


In [152]:
# Simulate binary outcomes for two groups
np.random.seed(42)
group1 = np.random.binomial(1, 0.04, 150)
weight1 = np.random.uniform(0.1, 2, 150)
weight1 = (
    weight1 / np.sum(weight1) * len(weight1)
)  # Normalize weights to have sum equal to n

group2 = np.random.binomial(1, 0.07, 100)
weight2 = np.random.uniform(0.1, 2, 100)
weight2 = (
    weight2 / np.sum(weight2) * len(weight2)
)  # Normalize weights to have sum equal to n

### t test

Interpretation

This is a test for the null hypothesis that two independent samples have identical average (expected) values.

If the p-value is greater than the common significance level of 0.05  
Then no evidence to reject null hypo of equal mean  
This means no enough evidence to conclude that the mean significantly differs

### test parameters

In [67]:
# 2. Welch's t-test (does not assume equal variance)
ttest_ind(group1, group2, alternative="less", equal_var=False)

TtestResult(statistic=np.float64(-1.2641828110871942), pvalue=np.float64(0.10396692542715495), df=np.float64(166.26029259354615))

In [68]:
# independent
ttest_ind(group1, group2, alternative="less", equal_var=True)

TtestResult(statistic=np.float64(-1.3470835706597655), pvalue=np.float64(0.08959144774244233), df=np.float64(248.0))

### variance assumptions

bernoulli rv paras:
- mean = p
- variance = p(1-p)
- std = sqrt(p(1-p))

Then for sample of size n,

X_bar, sample mean is a rv with:
- mean = p
- variance = p(1-p)/n
- std = sqrt(p(1-p)/n)

S^2, sample variance is a rv with:
- mean = p(1-p)
- variance = [p(1-p)]^2 * 2/(n-1)
- std = sqrt([p(1-p)]^2 * 2/(n-1)) = p(1-p) * sqrt(2/(n-1))

Also, Bessel's correction need to be applied to sample variance to arrive at an unbiased estimator.
- Unbiased variance estimator: s²_unbiased = Σ(xᵢ - x̄)² / (n-1) = s²_biased × n/(n-1)
- Unbiased std estimator: s_unbiased = √[s²_unbiased] = √[s²_biased × n/(n-1)]
- mean estimator: X_bar = Σxᵢ / n, which is unbiased since sample mean is an unbiased estimator of the population mean.
- s²_biased's special form for bernoulli rv: 
    - (xᵢ - x̄)² = (1 - X̄)² for xᵢ=1, and (xᵢ - x̄)² = X̄² for xᵢ=0, also X̄ = k/n
    - s²_biased = [k(1-X̄)² + (n-k)X̄²]/n = [k - 2kX̄ + kX̄² + nX̄² - kX̄²]/n = [k - 2kX̄ + nX̄²]/n = X̄ - 2X̄² + X̄² = X̄(1-X̄)
    - so s²_biased = x_bar * (1 - x_bar)


In [160]:
np.std(group1, ddof=1)

np.float64(0.19661566092456972)

In [161]:
def bernoulli_sample_std(values):
    p = np.mean(values)
    return np.sqrt(p * (1 - p) * len(values) / (len(values) - 1))


bernoulli_sample_std(group1)

np.float64(0.1966156609245697)

In [154]:
np.mean(group1), np.average(group1, weights=weight1)

(np.float64(0.04), np.float64(0.038803873811712124))

In [158]:
np.mean(group2), np.average(group2, weights=weight2)

(np.float64(0.08), np.float64(0.07140725805097317))

In [194]:
def effective_sample_size(weights):
    sum_w = np.sum(weights)
    sum_w2 = np.sum(weights**2)
    return (sum_w**2) / sum_w2


def weighted_unbiased_sample_std(values, weights):
    x_bar_weighted = np.average(values, weights=weights)
    biased_weighted_var = np.average((values - x_bar_weighted) ** 2, weights=weights)

    # Apply Bessel's correction, via kish's effective sample size
    effective_n = effective_sample_size(weights)
    print(f"{effective_n = }, n = {len(values)}")
    unbiased_var = biased_weighted_var * effective_n / (effective_n - 1)

    return np.sqrt(unbiased_var)

In [174]:
np.std(group1, ddof=1), weighted_unbiased_sample_std(group1, weight1)

effective_n = np.float64(119.09555157088872), n = 150


(np.float64(0.19661566092456972), np.float64(0.19394319610929034))

In [175]:
np.std(group2, ddof=1), weighted_unbiased_sample_std(group2, weight2)

effective_n = np.float64(76.29277781011024), n = 100


(np.float64(0.27265992434429076), np.float64(0.2592082826967816))

In [176]:
ttest_ind(
    group1,
    group2,
    equal_var=True,
    alternative="less",
)

TtestResult(statistic=np.float64(-1.3470835706597655), pvalue=np.float64(0.08959144774244233), df=np.float64(248.0))

In [177]:
ttest_ind_from_stats(
    np.mean(group1),
    np.std(group1, ddof=1),
    len(group1),
    np.mean(group2),
    np.std(group2, ddof=1),
    len(group2),
    equal_var=True,
    alternative="less",
)

Ttest_indResult(statistic=np.float64(-1.3470835706597653), pvalue=np.float64(0.08959144774244236))

In [193]:
ttest_ind_from_stats(
    np.average(group1, weights=weight1),
    np.std(group1, ddof=1),
    np.sum(weight1),
    np.average(group2, weights=weight2),
    np.std(group2, ddof=1),
    np.sum(weight2),
    equal_var=True,
    alternative="less",
)

Ttest_indResult(statistic=np.float64(-1.0979870814154022), pvalue=np.float64(0.13663750236136102))

In [195]:
ttest_ind_from_stats(
    np.average(group1, weights=weight1),
    weighted_unbiased_sample_std(group1, weight1),
    effective_sample_size(weight1),
    np.average(group2, weights=weight2),
    weighted_unbiased_sample_std(group2, weight2),
    effective_sample_size(weight2),
    equal_var=True,
    alternative="less",
)

effective_n = np.float64(119.09555157088872), n = 150
effective_n = np.float64(76.29277781011024), n = 100


Ttest_indResult(statistic=np.float64(-1.0030809844752682), pvalue=np.float64(0.15853761365364008))

In [199]:
# Using statsmodels for survey-weighted statistics
from statsmodels.stats.weightstats import DescrStatsW, CompareMeans

# Create weighted descriptive statistics
desc1 = DescrStatsW(group1, weights=weight1, ddof=1)
desc2 = DescrStatsW(group2, weights=weight2, ddof=1)

# Weighted t-test
cm = CompareMeans(desc1, desc2)
result = cm.ttest_ind(alternative="smaller", usevar="pooled")
print(f"Weighted t-test: {result}")

Weighted t-test: (np.float64(-1.1374439720607135), np.float64(0.1282252398412474), np.float64(248.0))


In [200]:
desc1.mean, desc1.std, desc1.nobs

(np.float64(0.038803873811712124),
 np.float64(0.19377424036866508),
 np.float64(150.0))